# DB 초기화 프로그램

In [1]:
!pip install pymysql > /dev/null

In [2]:
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving mysql.json to mysql.json


In [3]:
import json, pymysql
with open(filename) as fp:
    config_str = fp.read()
config = json.loads(config_str)

conn = pymysql.connect(**config)    # dictionary unpacking

In [4]:
import random
random.seed(2021)

## products 테이블

In [5]:
# products 테이블 생성 sql
sql = """
    CREATE TABLE if NOT exists products(
        pid INT PRIMARY KEY AUTO_INCREMENT,
        pname VARCHAR(40) NOT NULL,
        pprice INT UNSIGNED NOT NULL,
        pcategory VARCHAR(20) NOT NULL,
        pcost INT UNSIGNED NOT NULL
    ) AUTO_INCREMENT=11;
"""

In [6]:
cur = conn.cursor()
cur.execute(sql)

0

In [7]:
camping_goods = ['텐트','타프','천막','침낭','매트','랜턴','취사용품',
                 '아이스박스','보조배터리','워터저그']
car_goods = ['블랙박스','내비게이션','하이패스','후방카메라','자동차TV',
             '카오디오','핸즈프리','헤드유닛','방진매트','윤활방청제']
health_goods = ['러닝머신','웨이트기구','복근운동기구','벨트','거꾸리',
                '훌라후프','트램펄린','스텝퍼','로잉머신','헬스사이클']
category_names = ['캠핑용품','자동차용품','헬스용품']

In [11]:
for item in camping_goods:
    price = random.randint(100,999) * 100
    rate = random.randint(85, 98)
    cost = int(round(price * rate / 1000) * 10)
    print(item, price, '캠핑', cost)

텐트 94100 캠핑 83750
타프 15400 캠핑 13710
천막 17800 캠핑 15310
침낭 27900 캠핑 26230
매트 56100 캠핑 53860
랜턴 32500 캠핑 30550
취사용품 24000 캠핑 22800
아이스박스 53900 캠핑 49050
보조배터리 89500 캠핑 79660
워터저그 54600 캠핑 47500


In [9]:
sql = "insert into products(pname,pprice,pcategory,pcost) values(%s,%s,%s,%s);"
for i, item_list in enumerate([camping_goods, car_goods, health_goods]):
    for item in item_list:
        price = random.randint(100,999) * 100
        rate = random.randint(85, 98)
        cost = int(round(price * rate / 1000) * 10)
        cur.execute(sql, (item, price, category_names[i], cost))
conn.commit()

## sales 테이블

In [13]:
# spid를 외래키(foreign key)로 설정
sql = """
    CREATE TABLE if NOT EXISTS sales(
        sid INT PRIMARY KEY AUTO_INCREMENT,
        sdate DATE NOT NULL,
        scompany VARCHAR(20) NOT NULL,
        spid INT NOT NULL,
        sunit INT NOT NULL,
        FOREIGN KEY(spid) REFERENCES products(pid)
    ) AUTO_INCREMENT=101;
"""

In [14]:
cur = conn.cursor()
cur.execute(sql)

0

In [15]:
calendar = {1:31,2:29,3:31,4:30,5:31,6:30,7:31,8:31,9:30,10:31,11:30,12:31}
company_list = ['멀티캠퍼스','한경아카데미','표준협회','생산성본부','테크브루']
for i in range(10):
    date = f'2020-1-{random.randint(1,31)}'
    company = company_list[random.randint(0,4)]
    pid = random.randint(11,40)
    unit = random.randint(1,20)
    print(date,company,pid,unit)

2020-1-18 멀티캠퍼스 28 9
2020-1-30 멀티캠퍼스 28 19
2020-1-24 생산성본부 16 4
2020-1-2 멀티캠퍼스 24 11
2020-1-2 테크브루 23 17
2020-1-31 생산성본부 22 18
2020-1-8 표준협회 30 17
2020-1-9 표준협회 16 14
2020-1-19 표준협회 27 13
2020-1-6 테크브루 21 3


In [17]:
sql = "insert into sales values(default,%s,%s,%s,%s);"
for month, day in calendar.items():
    for i in range(10):
        date = f'2020-{month}-{random.randint(1,day)}'
        company = company_list[random.randint(0,4)]
        pid = random.randint(11,40)
        unit = random.randint(1,20)
        cur.execute(sql, (date, company, pid, unit))
conn.commit()

## 두 테이블을 Join 한 View 생성

In [18]:
sql = """
    CREATE VIEW sales_book as
        SELECT sid, sdate, scompany, pid, pname, pcategory, pprice, sunit, pcost,
            pprice*sunit AS revenue, (pprice-pcost)*sunit AS profit
        FROM sales
        JOIN products
        ON sales.spid = products.pid
        ORDER BY sdate;
"""

In [19]:
cur = conn.cursor()
cur.execute(sql)

0

- 데이터를 sales_book.csv로 저장

In [25]:
sql = 'select * from sales_book'
cur.execute(sql)
results = cur.fetchall()

In [26]:
import pandas as pd

df = pd.DataFrame(results)
df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,101,2020-01-06,한경아카데미,11,텐트,캠핑용품,94900,20,80660,1898000,284800
1,106,2020-01-06,표준협회,17,취사용품,캠핑용품,57800,1,49710,57800,8090
2,108,2020-01-07,생산성본부,29,방진매트,자동차용품,65500,3,57640,196500,23580
3,109,2020-01-10,한경아카데미,24,후방카메라,자동차용품,50400,17,48380,856800,34340
4,107,2020-01-12,표준협회,40,헬스사이클,헬스용품,47600,2,45700,95200,3800


In [27]:
del df[3]
del df[0]

In [28]:
df.columns = ['판매일자','판매처','상품명','카테고리','단가','수량','원가','매출','이익']
df.head()

,판매일자,판매처,상품명,카테고리,단가,수량,원가,매출액,이익
0,2020-01-06,한경아카데미,텐트,캠핑용품,94900,20,80660,1898000,284800
1,2020-01-06,표준협회,취사용품,캠핑용품,57800,1,49710,57800,8090
2,2020-01-07,생산성본부,방진매트,자동차용품,65500,3,57640,196500,23580
3,2020-01-10,한경아카데미,후방카메라,자동차용품,50400,17,48380,856800,34340
4,2020-01-12,표준협회,헬스사이클,헬스용품,47600,2,45700,95200,3800


In [29]:
df.to_csv('sales_book.csv', index=False)

## 마무리

In [30]:
cur.close()
conn.close()